# Import dependencies

In [1]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.cluster import KMeans, AffinityPropagation
import warnings
#warnings.filterwarnings("ignore")
import numpy as np

import torch
from torch.utils.tensorboard import SummaryWriter

from utils import clustering_classification, test_classifier, write, save_metrics_to_dict, encode_categorical_features, import_dataset, agg_clustering, writer_add_scalars

# For logging 

In [2]:
writer = SummaryWriter(log_dir="C:\\awilde\\britta\\LTU\\AppliedAI\\runs")
#writer = SummaryWriter()
metrics_dict={
"train_acc_dict" : {},
"train_f1_dict" : {},
"test_acc_dict" : {},
"test_f1_dict" : {},
"train_acc_avg" : {},
"train_f1_avg" : {},
"test_acc_avg" : {},
"test_f1_avg" : {},
}

In [3]:
# Launch tensorboard
# images=21 change this to how many datasets you use
%tensorboard --logdir runs/ --port 6006 --samples_per_plugin images=22
# If in use (Mac) use to find the process PID
% lsof -i :6006
# Kill the process with 
% kill -9 <PID>
# Then launch using bash with first command

UsageError: Line magic function `%tensorboard` not found.


# Config Params

In [4]:
TEST_SIZE = 0.2
RANDOM_SEEDS = [41, 42, 43, 44, 45]
K_FOLDS = 5

# Our chosen datasets
TODO
We have chosen the datasets based on the following criteria:
- datasets where we do not need to do any special preprocessing so that it is easy to do in only one pipeline
- rather small datasets to ensure we do not need high computational power

In [5]:
# Dataset that seems useful
dataset_id = {
    # "iris": 53, 
    # "heart_disease": 45, 
    # "wine_quality": 186, 
    # "breast_cancer_wisconsin_diagnostic": 17, 
    # "car_evaluation": 19, 
    # "spect_heart" : 95, 
    # "spectf_heart" : 96,
    # "mushroom": 73, 
    # "statlog" : 144, 
    # "credit_approval" : 27, 
    # "zoo" : 111, 
    # "balance_scale" : 12, 
    # "ilpd" : 225, 
    # "acute_inflamations" : 184, 
    # "ecoli" : 39, 
    "mammographic_mass" : 161, 
    # "hayes_roth" : 44, 
    # "habermans_survival" : 43, 
    # "congress_voting_records" : 105, 
    # "balloons" : 13, 
    # "lenses" : 58, 
    # "fertility" : 244, 
}

# sort alphabetically and adds id for logging
data_set_sorted = {}
for i, name in enumerate(sorted(dataset_id.keys())):
    data_set_sorted[name] = (dataset_id[name], i+1)

print(data_set_sorted)

{'mammographic_mass': (161, 1)}


# Import and preprocess datasets
For the preprocessing we will do the following steps:
1. Remove any missing values. In the article the following is written: "Given that our classifiers are not oriented to data with missing features, the missing inputs are treated as zero, which should not bias the comparison results." We therefore also decided to just remove missing values and to more focus on the full pipeline instead of single datasets. Another way could have been interpolation.
2. Encode categorical data into numerical data. This we have to do to use the classifiers later on.
3. Remove certain columns if they are highly correlated to others. <span style="color: red;">ALERT!</span>
4. Split the data into a train and a test set. We will use a 80/20 split.
5. Scale the data so that we have zero mean and standard deviation of one. This is done with the Standard scaler.

# Testing different classifiers

## K-Nearest Neighbor classifier

In [6]:
# https://towardsdatascience.com/gridsearchcv-for-beginners-db48a90114ee
knn_params = [{'knn__n_neighbors': [3, 5, 7, 9],
        'knn__weights': ['uniform', 'distance'],
        'knn__leaf_size': [15, 20]}]

## Support vector machine classifier

In [7]:
# params taken from here: https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/
svm_params = [{'svm__C': [0.1, 1, 10, 100, 1000],  
        'svm__gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
        'svm__kernel': ['rbf']} ]

## Logistic regression classifier

In [8]:
# params taken from here: https://www.geeksforgeeks.org/how-to-optimize-logistic-regression-performance/
# and from here https://www.kaggle.com/code/enespolat/grid-search-with-logistic-regression
log_reg_params = [
    {'log_reg__penalty':['l1','l2'],
    'log_reg__C' : np.logspace(-3,3,7),
    'log_reg__max_iter'  : [100,1000,2500,5000]
}
]

## Random Forest classifier

In [9]:
# https://www.datacamp.com/tutorial/random-forests-classifier-python
random_forest_params = [
    {"random_forest__n_estimators": [100, 500],
     "random_forest__max_depth" : [5, 10, 15]
}
]

## Gaussian naive bayes classifier

In [10]:
# https://www.kaggle.com/code/akshaysharma001/naive-bayes-with-hyperpameter-tuning#Hyperparameter-Tuning-to-improve-Accuracy
gnb_params = [
    {'gnb__var_smoothing': np.logspace(0,-9, num=10)
    }
]

## K-means unsupervised classifier

In [11]:
# Already use k-means++ and we set the no. clusters to no. of labels
# Set params in loop
kmeans_params = []

## Affinity propagation unsupervised classifier

In [12]:
# https://letsdatascience.com/affinity-propagation-clustering/
affinity_propagation_params = [
    # {"damping": [0.5, 0.7]}, 
    # #{"preference": [-50,-10,0,10,50]},
    # {"max_iter": [10000]}
    {"damping": [0.5, 0.7]}, 
    {"preference": [-50,-10,0,10,50]}
          ]

# Train test loop

In [13]:
for i, (name, id) in enumerate(data_set_sorted.items()):
        print("\n" + "#"*100)
        print(f"Current dataset: {name}")
        ordinal_encoder = OrdinalEncoder()

        # Set up dataset
        X, y = import_dataset(dataset_id[name], ordinal_encoder)
        labels = np.unique(y)
        y = encode_categorical_features(y, ordinal_encoder)
        print(f"Dataset size: {len(X)}")
        print(f"Labels in dataset: {labels}")
        print("#"*100, end="\n\n")


        for RANDOM_SEED in RANDOM_SEEDS:   

                print("*"*100)
                print("Current seed:", RANDOM_SEED)
                print("*"*100, end="\n\n")

                # split the dataset
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify = y)

                # # KNN
                # clf_name = "knn"
                # print("_"*100)
                # print(f"Classifier: {clf_name}")
                # # https://towardsdatascience.com/gridsearchcv-for-beginners-db48a90114ee
                # knn = KNeighborsClassifier()
                # train_acc, train_f1, test_acc, test_f1, cm_train, cm_test = test_classifier(knn, clf_name, knn_params, X_train, y_train, X_test, y_test)

                # write(writer, name, clf_name, cm_train, "train", id[1] )
                # write(writer, name, clf_name, cm_test, "test", id[1] )

                # metrics_dict = save_metrics_to_dict(clf_name,
                #                                 train_acc, 
                #                                 train_f1, 
                #                                 test_acc, 
                #                                 test_f1, 
                #                                 metrics_dict,
                #                                 i,
                #                                 RANDOM_SEED
                #                                 )
                
                # # SVM 
                # svm = SVC()
                # clf_name = "svm"
                # print("_"*100)
                # print(f"Classifier: {clf_name}")
                # train_acc, train_f1, test_acc, test_f1, cm_train, cm_test = test_classifier(svm, clf_name, svm_params, X_train, y_train, X_test, y_test)
                # write(writer, name, clf_name, cm_train, "train", id[1] )
                # write(writer, name, clf_name, cm_test, "test", id[1] )
                # metrics_dict = save_metrics_to_dict(clf_name,
                #                                 train_acc, 
                #                                 train_f1, 
                #                                 test_acc, 
                #                                 test_f1, 
                #                                 metrics_dict,
                #                                 i,
                #                                 RANDOM_SEED
                #                                 )

                # # Logistic regression
                # log_reg = LogisticRegression()
                # clf_name = "log_reg"
                # print("_"*100)
                # print(f"Classifier: {clf_name}")
                # train_acc, train_f1, test_acc, test_f1, cm_train, cm_test = test_classifier(log_reg, 
                #                                                                         clf_name, 
                #                                                                         log_reg_params, 
                #                                                                         X_train, 
                #                                                                         y_train, 
                #                                                                         X_test, 
                #                                                                         y_test)
                # write(writer, name, clf_name, cm_train, "train", id[1] )
                # write(writer, name, clf_name, cm_test, "test", id[1] )
                # metrics_dict = save_metrics_to_dict(clf_name,
                #                                 train_acc, 
                #                                 train_f1, 
                #                                 test_acc, 
                #                                 test_f1, 
                #                                 metrics_dict,
                #                                 i,
                #                                 RANDOM_SEED
                #                                 )
                
                # # Random forest
                # random_forest = RandomForestClassifier()
                # clf_name = "random_forest"
                # print("_"*100)
                # print(f"Classifier: {clf_name}")
                # train_acc, train_f1, test_acc, test_f1, cm_train, cm_test = test_classifier(random_forest, 
                #                                                                         clf_name, 
                #                                                                         random_forest_params, 
                #                                                                         X_train, 
                #                                                                         y_train, 
                #                                                                         X_test, 
                #                                                                         y_test)
                # write(writer, name, clf_name, cm_train, "train", id[1] )
                # write(writer, name, clf_name, cm_test, "test", id[1] )
                # metrics_dict = save_metrics_to_dict(clf_name,
                #                                 train_acc, 
                #                                 train_f1, 
                #                                 test_acc, 
                #                                 test_f1, 
                #                                 metrics_dict,
                #                                 i,
                #                                 RANDOM_SEED
                #                                 )
                
                # # Gaussian naive bayes
                # gnb = GaussianNB()
                # clf_name = "gnb"
                # print("_"*100)
                # print(f"Classifier: {clf_name}")
                # train_acc, train_f1, test_acc, test_f1, cm_train, cm_test = test_classifier(gnb, 
                #                                                                         clf_name, 
                #                                                                         gnb_params, 
                #                                                                         X_train, 
                #                                                                         y_train, 
                #                                                                         X_test, 
                #                                                                         y_test)
                # write(writer, name, clf_name, cm_train, "train", id[1] )
                # write(writer, name, clf_name, cm_test, "test", id[1] )
                # metrics_dict = save_metrics_to_dict(clf_name,
                #                                 train_acc, 
                #                                 train_f1, 
                #                                 test_acc, 
                #                                 test_f1, 
                #                                 metrics_dict,
                #                                 i,
                #                                 RANDOM_SEED
                #                                 )
                
        
                # # K-means
                # clf_name = "kmeans"
                # print("_"*100)
                # print(f"Classifier: {clf_name}")
                # kmeans_params = [{"algorithm": ["lloyd", "elkan"]}]
                # n_clusters = len(labels)
                # kmeans_params.append({"n_clusters": [n_clusters]})
                # train_acc, train_f1, test_acc, test_f1, cm_train, cm_test = clustering_classification(KMeans, clf_name, kmeans_params, X_train, y_train, X_test, y_test, RANDOM_SEED, K_FOLDS)
                # write(writer, name, clf_name, cm_train, "train", id[1])
                # write(writer, name, clf_name, cm_test, "test", id[1] )
                # metrics_dict = save_metrics_to_dict(clf_name,
                #                                 train_acc, 
                #                                 train_f1, 
                #                                 test_acc, 
                #                                 test_f1, 
                #                                 metrics_dict,
                #                                 i,
                #                                 RANDOM_SEED
                #                                 )

                # # Agglomerative clustering
                # clf_name = "agglomerative_clustering"
                # print("_"*100)
                # print(f"Classifier: {clf_name}")
                # train_acc, train_f1, test_acc, test_f1, cm_train, cm_test = agg_clustering(X_train, y_train, X_test, y_test, RANDOM_SEED)
                # write(writer, name, clf_name, cm_train, "train", id[1] )
                # write(writer, name, clf_name, cm_test, "test", id[1] )
                # metrics_dict = save_metrics_to_dict(clf_name,
                #                                 train_acc, 
                #                                 train_f1, 
                #                                 test_acc, 
                #                                 test_f1, 
                #                                 metrics_dict,
                #                                 i,
                #                                 RANDOM_SEED
                #                                 )
                

                # Affinity propagation
                clf_name = "affinity_propagation"
                print("_"*100)
                print(f"Classifier: {clf_name}")
                train_acc, train_f1, test_acc, test_f1, cm_train, cm_test =  clustering_classification(AffinityPropagation, clf_name, affinity_propagation_params, X_train, y_train, X_test, y_test, RANDOM_SEED, K_FOLDS)
                write(writer, name, clf_name, cm_train, "train", id[1] )
                write(writer, name, clf_name, cm_test, "test", id[1] )
                metrics_dict = save_metrics_to_dict(clf_name,
                                                train_acc, 
                                                train_f1, 
                                                test_acc, 
                                                test_f1, 
                                                metrics_dict,
                                                i,
                                                RANDOM_SEED
                                                )

        # Take average
        keys_to_update = ["train_acc_dict", "test_acc_dict", "train_f1_dict", "test_f1_dict"]
        for key in keys_to_update:
                metrics_dict[key] = {k: v / len(RANDOM_SEEDS) for k, v in metrics_dict[key].items()}
        # Write metrics to tensorboard, step is dataset id      
        writer.add_scalars("Train accuracy", metrics_dict["train_acc_dict"], id[1])
        writer.add_scalars("Test accuracy", metrics_dict["test_acc_dict"], id[1])
        writer.add_scalars("Train f1",  metrics_dict["train_f1_dict"], id[1])
        writer.add_scalars("Test f1", metrics_dict["test_f1_dict"], id[1])

        writer_add_scalars("Train average accuracy", writer, metrics_dict["train_acc_avg"], id[1])
        writer_add_scalars("Test average accuracy", writer, metrics_dict["test_acc_avg"], id[1])
        writer_add_scalars("Train average f1", writer, metrics_dict["train_f1_avg"], id[1])
        writer_add_scalars("Test average f1", writer, metrics_dict["test_f1_avg"], id[1])
        
        # reset dictionaries
        keys_to_update = ["train_acc_dict", "test_acc_dict", "train_f1_dict", "test_f1_dict"]
        for key in keys_to_update:
                metrics_dict[key] = {}
        

writer.close()


####################################################################################################
Current dataset: mammographic_mass
Dataset size: 830
Labels in dataset: [0 1]
####################################################################################################

****************************************************************************************************
Current seed: 41
****************************************************************************************************

____________________________________________________________________________________________________
Classifier: affinity_propagation
41
41
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 14 16 17 21 22 23 24 25 26 28 29
 32 33 34 36 39 42 43 45 47 48 55 58 59 60 62 63 64 66 68 70 74 77 79]
labels_map_cv keys: dict_keys([np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(14

c:\Users\britt\anaconda3\envs\AppliedAILabs\Lib\site-packages\sklearn\cluster\_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


[0 1 2 3 4 5 6 7 8 9]
labels_map_cv keys: dict_keys([np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)])
valid_clusters: [0 7 6 4 5 6 0 0 0 0 9 6 1 4 3 9 6 1 6 6 4 1 5 1 4 0 6 6 1 2 4 5 9 6 5 4 1
 2 4 1 2 5 1 1 4 4 4 1 7 4 3 5 4 7 1 6 5 4 5 0 0 0 1 0 6 6 1 0 4 4 4 4 4 6
 6 4 4 4 1 9 3 6 4 6 4 1 0 5 0 0 6 6 0 5 4 6 0 5 5 5 4 0 4 5 4 5 1 4 1 9 7
 1 6 6 8 0 9 6 6 5 0 0 3 0 7 4 6 6 6 1 4 4 0]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 21]
labels_map_cv keys: dict_keys([np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(20), np.int64(21)])
valid_clusters: [ 4  6 14 16  1  2  1  0 17 16  2  2 14 14  4  2  1  6  4 18 14 18 14  2
  6  6  6  6  4  6  6  1 17  1  6  2  7  1 20  1  6 14  1 1

c:\Users\britt\anaconda3\envs\AppliedAILabs\Lib\site-packages\sklearn\cluster\_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(
c:\Users\britt\anaconda3\envs\AppliedAILabs\Lib\site-packages\sklearn\cluster\_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


[0 1 2 3 4 5 6 7]
labels_map_cv keys: dict_keys([np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7)])
valid_clusters: [6 2 3 6 2 3 6 3 1 5 2 2 3 2 2 6 5 6 6 5 2 6 0 2 2 5 0 5 1 1 6 3 2 2 2 0 0
 2 6 0 2 6 5 2 5 2 4 1 5 1 5 4 1 1 1 6 5 5 0 5 1 1 0 6 1 1 1 2 2 6 6 1 2 1
 6 3 2 5 1 5 1 0 2 5 1 2 2 0 2 0 4 0 6 2 2 5 1 2 6 6 5 2 1 1 1 6 3 5 5 1 6
 5 2 2 2 6 6 2 6 2 6 2 5 6 2 2 6 2 6 2 3 1 1]
[ 0  1  2  3  4  5  6  8 10 11 12 14 17 18 19 21 22 24 25 26 28 29 32 33
 36 40 41 42 44 47 49 51 52 54 55 57 60 62]
labels_map_cv keys: dict_keys([np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(8), np.int64(10), np.int64(11), np.int64(12), np.int64(14), np.int64(17), np.int64(18), np.int64(19), np.int64(21), np.int64(22), np.int64(24), np.int64(25), np.int64(26), np.int64(28), np.int64(29), np.int64(32), np.int64(33), np.int64(36), np.int64(40), np.int64(41), np.int64(42), np.int64(44), np.int64(47),

...

labels_map_cv keys: dict_keys([np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(53), np.int64(54), np.int64(55), np.int64(56), np.int64(58), np.int64(59), np.int64(60), np.int64(61), np.int64(62), np.int64(63), np.int64(64), np.int64(65), np.int64(66), np.int64(67), np.int64(69), np.int64(70), np.int64(71),

c:\Users\britt\anaconda3\envs\AppliedAILabs\Lib\site-packages\sklearn\cluster\_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


labels_map_cv keys: dict_keys([np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(53), np.int64(54), np.int64(55), np.int64(56), np.int64(57), np.int64(58), np.int64(59), np.int64(60), np.int64(61), np.int64(62), np.int64(63), np.int64(64), np.int64(65), np.int64(66), np.int64(67), np.int64(68), np.int64(69),

c:\Users\britt\anaconda3\envs\AppliedAILabs\Lib\site-packages\sklearn\cluster\_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  58  59  60  61  62  63  64  65  66  67  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  95  96  97  98 100 101 102 104 105 106 107 109 110 111 112 114
 115 116 118 119 120 121 122 123 124 125 126 127 128 129 130 132 133 134
 135 136 137 138 139 140 141 142 143 145 146 148 149 151 152 153 154 155
 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173
 174 175 176 177 178 179 181 182 183 185 186 187 188 189 191 192 193 195
 196 197 198 199 200 201 203 204 205 206 207 208 209 210 211 213 214 215
 216 217 218 220 222 223 224 225 227 228 229 230 231 232 233 234 235 236
 237 238 239 240 241 242 243 244 245 247 248 250 251 252 253 254 255 256
 257 258 259 260 261 262 263 264 265 266 267 269 27

In [14]:
rank_counter = 0
print(f"{'Rank':<5} {'Classifier':<30} {'Avg. test acc.':<17} {'Avg. train acc.':<17} {'Avg. test F1':<17} {'Avg train F1':<17}")
print("_"*100)
for i, clf in enumerate(sorted(metrics_dict["test_acc_avg"].items(), key=lambda x: x[1], reverse=True)):
    if clf[0].endswith("count"):
        rank_counter += 1
        continue
    print(f"{i+1-rank_counter:<5} {clf[0][:-4]:<30} {clf[1]:<17.3f} {metrics_dict['train_acc_avg'][clf[0]]:<8.3f}\
          {metrics_dict['test_f1_avg'][clf[0]]:<18.3f}{metrics_dict['train_f1_avg'][clf[0]]:<17.3f}")
print("_"*100)

Rank  Classifier                     Avg. test acc.    Avg. train acc.   Avg. test F1      Avg train F1     
____________________________________________________________________________________________________
1     affinity_propagation           0.771             0.830             0.771             0.830            
____________________________________________________________________________________________________
